In [303]:
import pandas as pd
import numpy as np
import processing.b_feats.util
import processing.processing_utils
import importlib
from constants import LEVELS, START
from compress_pickle import dump, load
import sys
import os
os.chdir('A:/ebay/')
sys.path.append('repo/processing/b_feats/test/')
import meta_time_feats_test  
importlib.reload(meta_time_feats_test)
importlib.reload(processing.b_feats.time_funcs)
importlib.reload(processing.b_feats.util)
setup_complex_lstg = meta_time_feats_test.setup_complex_lstg
events = setup_complex_lstg(None, meta=1, leaf=2)
events = setup_complex_lstg(events, meta=1, leaf=1, starter=11)
events.index = events.index.droplevel(['cndtn'])
feats = processing.b_feats.util.get_cat_time_feats(events, levels=['meta', 'leaf'])
feats.isna().any().any()

meta
['meta']
leaf
['meta', 'leaf']


False

In [304]:
feats

,meta_lstgs_open,meta_lstg_inds,meta_threads,meta_slr_offers,meta_byr_offers,meta_accepts,meta_accept_norm_25,meta_accept_norm_50,meta_accept_norm_75,meta_accept_norm_100,leaf_lstgs_open,leaf_lstg_inds,leaf_threads,leaf_slr_offers,leaf_byr_offers,leaf_accepts,leaf_accept_norm_25,leaf_accept_norm_50,leaf_accept_norm_75,leaf_accept_norm_100
lstg,,,,,,,,,,,,,,,,,,,,
1,2,21,32,22,31,16,0.5,0.7,0.85,1.0,1,10,14,9,13,8,0.5,0.7,0.85,1.00
2,4,21,34,24,33,15,0.5,0.7,0.85,1.0,2,10,16,11,15,7,0.5,0.7,0.85,1.00
3,6,21,35,25,34,15,0.5,0.8,0.85,1.0,3,10,17,12,16,7,0.5,0.8,0.85,1.00
4,6,21,35,26,35,15,0.6,0.8,0.85,1.0,3,10,17,13,17,7,0.6,0.8,0.85,1.00
5,2,21,34,26,34,15,0.5,0.8,0.85,1.0,1,10,16,13,16,7,0.5,0.8,0.85,1.00
6,4,21,34,25,34,15,0.6,0.8,0.85,1.0,2,10,16,12,16,7,0.6,0.8,0.85,1.00
7,6,21,34,26,35,15,0.5,0.7,0.85,1.0,3,10,16,13,17,7,0.5,0.7,0.80,0.95
8,2,21,35,24,35,15,0.5,0.7,0.85,1.0,1,10,17,11,17,7,0.5,0.7,0.80,1.00
9,4,21,35,25,35,15,0.5,0.7,0.85,1.0,2,10,17,12,17,7,0.5,0.7,0.80,1.00


In [196]:
actual

,meta_lstgs_open,meta_lstg_inds,meta_threads,meta_slr_offers,meta_byr_offers,meta_accepts,meta_accept_norm_25,meta_accept_norm_50,meta_accept_norm_75,meta_accept_norm_100
lstg,,,,,,,,,,
1,1,10,14,9,13,8,0.5,0.7,0.85,1.00
2,2,10,16,11,15,7,0.5,0.7,0.85,1.00
3,3,10,17,12,16,7,0.5,0.8,0.85,1.00
4,3,10,17,13,17,7,0.6,0.8,0.85,1.00
5,1,10,16,13,16,7,0.5,0.8,0.85,1.00
6,2,10,16,12,16,7,0.6,0.8,0.85,1.00
7,3,10,16,13,17,7,0.5,0.7,0.80,0.95
8,1,10,17,11,17,7,0.5,0.7,0.80,1.00
9,2,10,17,12,17,7,0.5,0.7,0.80,1.00


In [72]:
total_lstgs_index = pd.MultiIndex.from_tuples([(1, 2), (1, 1), (2, 3), (2, 1)], names=['meta','leaf'])
total_lstgs = pd.DataFrame(index=total_lstgs_index, data={'tot': [4, 3, 1, 9]})
lstg_index = pd.MultiIndex.from_tuples([(1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 1, 6), (1, 1, 7), (1, 1, 8), (1, 1, 9), (2, 3, 1), (2, 1, 3), (2, 1, 2)], names=['meta', 'leaf', 'lstg'])
lstg_index

MultiIndex([(1, 2, 1),
            (1, 2, 2),
            (1, 2, 3),
            (1, 1, 6),
            (1, 1, 7),
            (1, 1, 8),
            (1, 1, 9),
            (2, 3, 1),
            (2, 1, 3),
            (2, 1, 2)],
           names=['meta', 'leaf', 'lstg'])

In [73]:
total_lstgs.reindex(lstg_index)

tot
meta leaf lstg     
1    2    1       4
          2       4
          3       4
     1    6       3
          7       3
          8       3
          9       3
2    3    1       1
     1    3       9
          2       9

In [74]:
total_lstgs_index = pd.Index([1, 2])
total_lstgs = pd.DataFrame(index=total_lstgs_index, data={'tot': [4, 3]})
lstg_index = pd.MultiIndex.from_tuples([(1, 1), (1, 2), (1, 3), (2, 3), (2, 2)], names=['meta', 'lstg'])
lstg_index

MultiIndex([(1, 1),
            (1, 2),
            (1, 3),
            (2, 3),
            (2, 2)],
           names=['meta', 'lstg'])

In [75]:
total_lstgs.reindex(lstg_index, level='meta')

tot
meta lstg     
1    1       4
     2       4
     3       4
2    3       3
     2       3

In [77]:
t = pd.Index([1, 2, 3, 4])
p = pd.Index([2, 1, 3, 4])

In [78]:
t.equals(p)

False

In [107]:
t = pd.DataFrame(data={'clock': [4, 9, 11, 1, 2, 15, 15], 'censored': [2, 2, 2, 2, 2, 2, 1]})

In [109]:
t.sort_values(['clock', 'censored'])

,clock,censored
3,1,2
4,2,2
0,4,2
1,9,2
2,11,2
6,15,1
5,15,2


In [118]:
t

,clock,censored
0,4,2
1,9,2
2,11,2
3,1,2
4,2,2
5,15,2
6,15,1
